In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.animation as animation
pi = np.pi
data_L = []

In [2]:
# Utility Functions

from numpy import cross, eye, dot
from scipy.linalg import expm, norm

def Rot_Mat(axis, theta):
    return expm(cross(eye(3), axis/norm(axis)*theta))

def rotate_filament(filament, angle, axis_vector=np.array([0,0,1])):
    newpos = np.dot(Rot_Mat(axis_vector, angle), filament.position)
    newfilvec = np.dot(Rot_Mat(axis_vector, angle), filament.length_vector)
    return VortexFilament(filament.vorticity, newpos, newfilvec, filament.vortex_core_radius)

def rotate_vortex_array(vortex_array, angle, axis_vector=np.array([0,0,1])):
    new_vort_array = create_copy_vortex_array(vortex_array)
    for j in range(len(vortex_array.vortex_array)):
        for fil in vortex_array.get_vortex_array(j):
            new_vort_array.add_vortex_filament(rotate_filament(fil, angle, axis_vector),j)
    return new_vort_array

def create_copy_vortex_array(vortex_array):
    new_vort_array = FullVortexArray2(vortex_array.max_filaments)
    for arr in vortex_array.vortex_array:
        new_vort_array.vortex_array.append([fil.get_copy() for fil in arr])
    return new_vort_array


In [3]:
# Basic Classes

class VortexFilament:
    def __init__(self, vorticity, position, filament_length_vector, vortex_core_radius):
        self.vorticity = vorticity
        self.position = np.array(position)
        self.length_vector = filament_length_vector
        self.vortex_core_radius = vortex_core_radius
        self.update_endpoints()

    def get_induced_velocity(self, filament=None, abs_pos=None, rel_pos=None):
        """ Finds the induced velocity by current filament on another filament """
        """ ! vortex blow not fixed """
        rc = self.vortex_core_radius
        if filament != None:
            r = filament.position - self.position
        elif abs_pos != None:
            r = np.array(abs_pos) - self.position
        elif rel_pos != None:
            r = np.array(rel_pos)
        mr = np.linalg.norm(r)
        f = 1
        if mr<self.vortex_core_radius:
            f = mr**2/(self.vortex_core_radius**2)
        # yet to add vortex core effect
#         if mr==0:
#             print('mr', mr, r)
#             print(filament == self)
        ind_vel = self.vorticity/(4*pi)*(np.cross(self.length_vector, r))/(mr)**3*f
        #
        #ind_vel = self.vorticity/(4*pi)*(np.cross(r1,r2)*(mr1+mr2)*(1-np.dot(r1,r2)/(mr1*mr2)))/((mr1*mr2)**2-(np.dot(r1,r2)**2)+rc**2*(mr1**2+mr2**2-2*np.dot(r1,r2)))
        return ind_vel
    
    def get_induced_velocity2(self, filament=None, abs_pos=None, rel_pos=None):
        """ Finds the induced velocity by current filament on another filament """
        """ ! vortex blow not fixed """
        rc = self.vortex_core_radius
        if filament != None:
            r1 = filament.position - self.endpoints[0]
            r2 = filament.position - self.endpoints[1]
        elif abs_pos != None:
            r1 = np.array(abs_pos) - self.endpoints[0]
            r2 = np.array(abs_pos) - self.endpoints[1]
        elif rel_pos != None:
            r1 = np.array(rel_pos[0])
            r2 = np.array(rel_pos[1])
        mr1 = np.linalg.norm(r1)
        mr2 = np.linalg.norm(r2)
        ind_vel = self.vorticity/(4*pi)*(np.cross(r1,r2)*(mr1+mr2)*(1-np.dot(r1,r2)/(mr1*mr2)))/((mr1*mr2)**2-(np.dot(r1,r2)**2)+rc**2*(mr1**2+mr2**2-2*np.dot(r1,r2)))
        return ind_vel
        
    def update_endpoints(self):
        self.endpoints = [self.position - self.length_vector/2, self.position + self.length_vector/2]
        
    def update_pos(self, new_position):
        self.position = new_position
        self.update_endpoints()
        
    def get_copy(self):
        return VortexFilament(self.vorticity, self.position, self.length_vector, self.vortex_core_radius)
    
    def __add__(self, pos):
        return self.position + pos.pos

'''
class FullVortexArray:
  """ this contains the vortices 3 x K"""
  def __init__(self, max_no_of_filaments):
    self.vortex_array = []
    self.max_filaments = max_no_of_filaments

  def add_vortex_filament(self, vortex_filament):
    self.vortex_array.append(vortex_filament)
    self.vortex_array = self.vortex_array[-self.max_filaments:]
    
  def get_filament(self, psi, zeta_index):
    """ this container gives out r if phi, zeta are given as i/o """
    return rotate_filament(self.vortex_array[zeta_index], psi)

  def get_vortex_array(self, psi=0):
    """  """
    return rotate_vortex_array(self.vortex_array, psi).vortex_array

  def gen_copy_vort_array(self, psi=0):
    return [self.vortex_array[psi]

  def compute_ind_vel_vector(psi, by_zeta_index, at_zeta_index, by_arr=None):
    if by_arr == None:
      by_arr = self
    by_fil = by_arr.get_filament(psi, by_zeta_index)
    at_fil = self.get_filament(psi, at_zeta_index)
    return by_fil.get_induced_velocity(filament = at_fil)

  def compute_net_ind_vel(psi, at_zeta_index, other_vortex_arrays):
    """ Computes the net induced veolcity using Bio-Savart's law """
    
    ind_vel = np.zeros(3)
    for by_zeta_index in range(len(self.vortex_array)):
        if by_zeta_index != at_zeta_index:
            ind_vel += self.compute_ind_vel_vector(psi, by_zeta_index, at_zeta_index)
    for arr in other_vortex_arrays:
        for by_zeta_index in range(len(arr)):
            ind_vel += self.compute_ind_vel_vector(psi, by_zeta_index, at_zeta_index, arr)
    return ind_vel
'''

class FullVortexArray2:
    """ this is a list of the vortices 3 x K"""
    def __init__(self, max_no_of_filaments):
        self.vortex_array = []
        self.max_filaments = max_no_of_filaments

    def add_vortex_filament(self, vortex_filament, psi_index):
        if psi_index < len(self.vortex_array):
            self.vortex_array[psi_index].append(vortex_filament)
            self.vortex_array[psi_index] = self.vortex_array[psi_index][-self.max_filaments:]
            #print(len(self.vortex_array[-self.max_filaments:]))

    def add_filamnet_at_tip(self, blade_radius, psi_index, psi, tip_height=0):
        vorticity, vort_core_rad = self.vortex_array[psi_index][-1].vorticity, self.vortex_array[psi_index][-1].vortex_core_radius
        pos = np.array([blade_radius*np.cos(psi), blade_radius*np.sin(psi), tip_height])
        fil_len = np.linalg.norm(pos - self.get_filament(psi_index,-1).position)
        fil_len_vec = (np.cross(pos,[0,0,1])/np.linalg.norm(np.cross(pos,[0,0,1])))*fil_len
        #print(pos, fil_len_vec)
        #pos = pos - fil_len_vec/2
        self.add_vortex_filament(VortexFilament(vorticity, pos, fil_len_vec, vort_core_rad), psi_index)

    def add_vortex_array(self, vort_arr):
        self.vortex_array.append(vort_arr)

    def add_new_psi(self):
        self.vortex_array.append([])

    def update_filament_pos(self, psi_index, zeta_index, new_pos):
        self.get_filament(psi_index, zeta_index).update_pos(new_pos)
        
    def update_filament_vorticity(self, psi_index, zeta_index, new_v):
        self.get_filament(psi_index, zeta_index).vorticity = new_v

    def update_array_poses(self, psi_index, pos_arr):
        for zeta_index in range(len(self.get_vortex_array(psi_index))):
            self.update_filament_pos(psi_index, zeta_index, pos_arr[-(zeta_index+1)])

    def get_filament(self, psi_index, zeta_index):
        """ this container gives out r if phi, zeta are given as i/o """
        try:
            return self.vortex_array[psi_index][-(zeta_index+1)]
        except:
            print('yo', len(self.vortex_array))

    def get_vortex_array(self, psi_index=0):
        return self.vortex_array[psi_index]

    def get_copy(self):
        #print('HEEE')
        new_vort_array = FullVortexArray2(self.max_filaments)
        for arr in self.vortex_array:
            new_vort_array.vortex_array.append([fil.get_copy() for fil in arr])
        return new_vort_array

    def compute_ind_vel_vector(self, psi_index, by_zeta_index, at_zeta_index, by_arr=None):
        if by_arr == None:
            #print('hi')
            by_arr = self
        by_fil = by_arr.get_filament(psi_index, by_zeta_index)
        #print(by_fil)
        at_fil = self.get_filament(psi_index, at_zeta_index)
        return by_fil.get_induced_velocity2(filament = at_fil)

    def compute_net_ind_vel(self, psi_index, at_zeta_index, all_vortex_arrays):
        """ Computes the net induced velocity using Bio-Savart's law """

        ind_vel = np.zeros(3)
        for by_zeta_index in range(len(self.vortex_array[psi_index])):
            if by_zeta_index != at_zeta_index:
                #print(by_zeta_index, at_zeta_index)
                ind_vel += self.compute_ind_vel_vector(psi_index, by_zeta_index, at_zeta_index)
        cnt = 0
        for arr in all_vortex_arrays:
            if arr != self:
                cnt+=1
                for by_zeta_index in range(len(arr.vortex_array[psi_index])):
                    #print(by_zeta_index, at_zeta_index, cnt)
                    ind_vel += self.compute_ind_vel_vector(psi_index, by_zeta_index, at_zeta_index, arr)
        #print('END')
        return ind_vel

In [4]:
# Basic Functions

# def generate_prescribed_wake_vortex_array(theta_twist, C_T, num_fil):
#     R = 0.8255
#     CT = 0.0050351
#     SIGMA = 0.0979
#     b = 4
#     #num_fil = 200
#     phi_v = np.linspace(0,10*pi,num_fil)
#     Zv = np.zeros(len(phi_v))
#     TWIST = -13 #DEGREES
#     k1 = -0.25*(CT/SIGMA + 0.001*TWIST)
#     k2 = -(CT)**0.5 - 0.01*((CT)**0.5)*TWIST
#     for i in range(len(phi_v)):
#         if(phi_v[i] < 2*pi/b):
#             Zv[i] = R*(phi_v[i])*k1

#         elif(phi_v[i] > 2*pi/b):
#             Zv[i] = R*(k1*2*pi/b + k2*(phi_v[i] - 2*pi/b))
#     A = 0.78
#     lamda = 0.145 + 27*CT
#     Rv = np.zeros(len(phi_v))
#     for i in range(len(phi_v)):
#         Rv[i] = (A + (1 - A)*np.exp(-lamda*phi_v[i]))*R

#     Xv = Rv*np.cos(phi_v)
#     Yv = Rv*np.sin(phi_v)

#     pos_fil = np.zeros((num_fil,3))
#     pos_fil[:,0] = Xv
#     pos_fil[:,1] = Yv
#     pos_fil[:,2] = Zv

#     return(pos_fil)

def generate_prescribed_wake_vortex_array(theta_twist, C_T, num_tip_vortex):
    R = 0.8255
    CT = C_T
    SIGMA = 0.0979
    b = 4
    phi_v = np.linspace(0, 6*pi, num_tip_vortex) ### to change size of helix
    Zv = np.zeros(num_tip_vortex)
    num_fil = num_tip_vortex - 1
    TWIST = theta_twist
    k1 = -0.25*(CT/SIGMA + 0.001*TWIST)
    k2 = -(CT)**0.5 - 0.01*((CT)**0.5)*TWIST
    for i in range(num_tip_vortex):
        if(phi_v[i] < 2*pi/b):
            Zv[i] = R*(phi_v[i])*k1

        elif(phi_v[i] > 2*pi/b):
            Zv[i] = R*(k1*2*pi/b + k2*(phi_v[i] - 2*pi/b))
    A = 0.78
    lamda = 0.145 + 27*CT
    Rv = np.zeros(num_tip_vortex)
    for i in range(num_tip_vortex):
        Rv[i] = (A + (1 - A)*np.exp(-lamda*phi_v[i]))*R

    Xv = Rv*np.cos(-phi_v)
    Yv = Rv*np.sin(-phi_v)

    X_fil_mid = (Xv[1:] + Xv[:-1])/2
    Y_fil_mid = (Yv[1:] + Yv[:-1])/2
    Z_fil_mid = (Zv[1:] + Zv[:-1])/2

#     len_fil = ((Xv[1;:] - Xv[:,-1])**2 + (Yv[1;:] - Yv[:,-1])**2 + (Zv[1;:] - Zv[:,-1])**2)**0.5
    X_len_fil = -(Xv[1:] - Xv[:-1])
    Y_len_fil = -(Yv[1:] - Yv[:-1])
    Z_len_fil = -(Zv[1:] - Zv[:-1])
    
    fil_vec = np.zeros((num_fil,3))
    fil_vec[:,0] = X_len_fil
    fil_vec[:,1] = Y_len_fil
    fil_vec[:,2] = Z_len_fil
    
    pos_fil = np.zeros((num_fil,3))
    pos_fil[:,0] = X_fil_mid 
    pos_fil[:,1] = Y_fil_mid
    pos_fil[:,2] = Z_fil_mid
    print(np.arctan2(pos_fil[2][0], pos_fil[2][1]), np.arctan2(pos_fil[1][0], pos_fil[1][1]))
    
    return(pos_fil, fil_vec)


def create_mirrored_wake(vort_arr, grnd_ht = 1):
    mirr_arr = [va.get_copy() for va in vort_arr]
    for va in mirr_arr:
        for j in range(len(va.vortex_array)):
            for k in range(len(va.vortex_array[j])):
                va.update_filament_pos(j, k, va.get_filament(j,k).position + [0,0, -2*va.get_filament(j,k).position[-1] - 2*grnd_ht])
                va.update_filament_vorticity(j, k, -va.get_filament(j,k).vorticity)
                
    return mirr_arr

def create_mirrored_vort_arr(vort_arr, grnd_ht = 1):
    mirr_arr = [fil.get_copy() for fil in vort_arr]
    for k in range(len(mirr_arr)):
        mirr_arr[k].update_pos(vort_arr[k].position + [0,0, -2*vort_arr[k].position[-1] - 2*grnd_ht])
        mirr_arr[k].vorticity = -vort_arr[k].vorticity
    return mirr_arr


def solve_free_wake(initial_filament_arrays, omega, blade_radius, free_stream_vel, update_fn, conv_check_fn, time_conv_check_fn, time_step, max_time_steps=10, max_iters=10, grnd_ht=1):
    curr_arrs = initial_filament_arrays
    for i in range(max_iters):
        print("Iteration no. ", i)
        prev_arrs = [arr.get_copy() for arr in curr_arrs]
        #print('dine')
        #print(curr_arrs[0].vortex_array)
        pack = [time_step, max_time_steps, omega, blade_radius, free_stream_vel, i]
        if update_fn == free_wake_with_grnd_update:
            pack.append(grnd_ht)
        update_fn(prev_arrs, curr_arrs, time_conv_check_fn, pack)
        #print([[curr_arrs[0].vortex_array[j][k].position for k in range(10)] for j in range(10)])
        if conv_check_fn([prev_arrs, curr_arrs]):
            break
    return curr_arrs

            
def free_wake_update(prev_data, curr_data, time_conv_check_fn, addn_info):
    prev_arrs = prev_data
    curr_arrs = curr_data
    '''for bl in range(len(curr_arrs)):
        for k in range(len(curr_arrs[bl_arr_index].get_vortex_array(0))):
            print(curr_arrs[bl_arr_index].get_filament(0,k).position
    return'''
    temp_arrs = [arr.get_copy() for arr in curr_arrs]
    time_step, max_time_steps, omega, blade_radius, free_stream_vel, n_index = addn_info
    if n_index != 0:
        for j in range(max_time_steps-1):
#             print("J", j)
#             print([np.array(prev_arrs[3].get_vortex_array(j)[k].position) for k in range(4)])
            for bl_arr_index in range(len(temp_arrs)):
#                 print('bl', bl_arr_index)
                sum1 = np.array([prev_arrs[bl_arr_index].compute_net_ind_vel(j, zet_ind, prev_arrs) for zet_ind in range(len(prev_arrs[bl_arr_index].get_vortex_array(j)))])
                sum2 = np.array([prev_arrs[bl_arr_index].compute_net_ind_vel(j+1, zet_ind, prev_arrs) for zet_ind in range(len(prev_arrs[bl_arr_index].get_vortex_array(j)))])
                new_poses = time_step*(free_stream_vel+0.5*sum1+0.5*sum2)
                #print(new_poses)
                ###
                '''for k in range(10):
                    if (np.linalg.norm(new_poses[k])/np.linalg.norm(temp_arrs[bl_arr_index].get_filament(j,k).position))>0.1:
                        print('hi')'''
                ###
                for k in range(len(temp_arrs[bl_arr_index].get_vortex_array(j))):
                    new_poses[-(k+1),:] += temp_arrs[bl_arr_index].get_filament(j,k).position
                temp_arrs[bl_arr_index].update_array_poses(j+1, new_poses)
                temp_arrs[bl_arr_index].add_filamnet_at_tip(blade_radius, j+1, bl_arr_index*2*pi/(len(curr_arrs))+omega*(j+1)*time_step)
            for bl_arr_index in range(len(curr_arrs)):
                sum1 = np.array([temp_arrs[bl_arr_index].compute_net_ind_vel(j, zet_ind, temp_arrs) for zet_ind in range(len(temp_arrs[bl_arr_index].get_vortex_array(j)))])
                sum2 = np.array([temp_arrs[bl_arr_index].compute_net_ind_vel(j+1, zet_ind, temp_arrs) for zet_ind in range(len(temp_arrs[bl_arr_index].get_vortex_array(j)))])
                new_poses = time_step*(free_stream_vel+0.5*sum1+0.5*sum2)
                #print(new_poses)
                for k in range(len(curr_arrs[bl_arr_index].get_vortex_array(j))):
                    new_poses[-(k+1),:] += curr_arrs[bl_arr_index].get_filament(j,k).position
                curr_arrs[bl_arr_index].update_array_poses(j+1, new_poses)
                curr_arrs[bl_arr_index].add_filamnet_at_tip(blade_radius, j+1, bl_arr_index*2*pi/(len(curr_arrs))+omega*(j+1)*time_step)
    else:
        for j in range(max_time_steps-1):
#             print("J", j)
            for bl_arr_index in range(len(curr_arrs)):
#                 print('bl', bl_arr_index, bl_arr_index*2*pi/(len(curr_arrs)))
                sum1 = np.array([curr_arrs[bl_arr_index].compute_net_ind_vel(j, zet_ind, curr_arrs) for zet_ind in range(len(curr_arrs[bl_arr_index].get_vortex_array(j)))])
                #print(np.array(prev_arrs[bl_arr_index].get_vortex_array(j)), (time_step*(free_stream_vel+sum1)).shape)
                new_poses = time_step*(free_stream_vel+sum1)
                #print(new_poses)
                for k in range(len(curr_arrs[bl_arr_index].get_vortex_array(j))):
                    #print(new_poses[k,:] + curr_arrs[bl_arr_index].get_filament(j,k).position)
                    new_poses[-(k+1),:] += curr_arrs[bl_arr_index].get_filament(j,k).position
                curr_arrs[bl_arr_index].add_vortex_array(curr_arrs[bl_arr_index].get_copy().get_vortex_array(j))
                curr_arrs[bl_arr_index].update_array_poses(j+1, new_poses)
                curr_arrs[bl_arr_index].add_filamnet_at_tip(blade_radius, j+1, bl_arr_index*2*pi/(len(curr_arrs))+omega*(j+1)*time_step)
#                 print('end', len(curr_arrs[bl_arr_index].vortex_array))
        '''for bl in range(len(curr_arrs)):
            print('BL ', bl)
            for k in range(len(curr_arrs[bl].get_vortex_array(0))):
                print(curr_arrs[bl].get_filament(0,k).position, curr_arrs[bl].get_filament(1,k).position)'''


def free_wake_with_grnd_update(prev_data, curr_data, time_conv_check_fn, addn_info):
    time_step, max_time_steps, omega, blade_radius, free_stream_vel, n_index, grnd_ht = addn_info
    prev_arrs = prev_data
    curr_arrs = curr_data
    '''for bl in range(len(curr_arrs)):
        for k in range(len(curr_arrs[bl_arr_index].get_vortex_array(0))):
            print(curr_arrs[bl_arr_index].get_filament(0,k).position
    return'''
    temp_arrs = [arr.get_copy() for arr in curr_arrs]
    
    if n_index != 0:
        for j in range(max_time_steps-1):
#             print("J", j)
#             print([np.array(prev_arrs[3].get_vortex_array(j)[k].position) for k in range(4)])
            for bl_arr_index in range(int(len(temp_arrs)/2)):
#                 print('bl', bl_arr_index)
                sum1 = np.array([prev_arrs[bl_arr_index].compute_net_ind_vel(j, zet_ind, prev_arrs) for zet_ind in range(len(prev_arrs[bl_arr_index].get_vortex_array(j)))])
                sum2 = np.array([prev_arrs[bl_arr_index].compute_net_ind_vel(j+1, zet_ind, prev_arrs) for zet_ind in range(len(prev_arrs[bl_arr_index].get_vortex_array(j)))])
                new_poses = time_step*(free_stream_vel+0.5*sum1+0.5*sum2)
                #print(new_poses)
                ###
                '''for k in range(10):
                    if (np.linalg.norm(new_poses[k])/np.linalg.norm(temp_arrs[bl_arr_index].get_filament(j,k).position))>0.1:
                        print('hi')'''
                ###
                for k in range(len(temp_arrs[bl_arr_index].get_vortex_array(j))):
                    new_poses[-(k+1),:] += temp_arrs[bl_arr_index].get_filament(j,k).position
                temp_arrs[bl_arr_index].update_array_poses(j+1, new_poses)
                temp_arrs[bl_arr_index].add_filamnet_at_tip(blade_radius, j+1, bl_arr_index*2*pi/(len(curr_arrs))+omega*(j+1)*time_step)
                new_poses2 = [fil.position for fil in create_mirrored_vort_arr(temp_arrs[bl_arr_index].get_vortex_array(j+1), grnd_ht)]
                temp_arrs[int(len(temp_arrs)/2)+bl_arr_index].update_array_poses(j+1, new_poses2)
            for bl_arr_index in range(int(len(curr_arrs)/2)):
                sum1 = np.array([temp_arrs[bl_arr_index].compute_net_ind_vel(j, zet_ind, temp_arrs) for zet_ind in range(len(temp_arrs[bl_arr_index].get_vortex_array(j)))])
                sum2 = np.array([temp_arrs[bl_arr_index].compute_net_ind_vel(j+1, zet_ind, temp_arrs) for zet_ind in range(len(temp_arrs[bl_arr_index].get_vortex_array(j)))])
                new_poses = time_step*(free_stream_vel+0.5*sum1+0.5*sum2)
                #print(new_poses)
                for k in range(len(curr_arrs[bl_arr_index].get_vortex_array(j))):
                    new_poses[-(k+1),:] += curr_arrs[bl_arr_index].get_filament(j,k).position
                curr_arrs[bl_arr_index].update_array_poses(j+1, new_poses)
                curr_arrs[bl_arr_index].add_filamnet_at_tip(blade_radius, j+1, bl_arr_index*2*pi/(len(curr_arrs))+omega*(j+1)*time_step)
                new_poses2 = [fil.position for fil in create_mirrored_vort_arr(curr_arrs[bl_arr_index].get_vortex_array(j+1), grnd_ht)]
                curr_arrs[int(len(curr_arrs)/2)+bl_arr_index].update_array_poses(j+1, new_poses2)
    else:
        prev_arrs = prev_data + create_mirrored_wake(prev_data, grnd_ht)
        curr_arrs = curr_data + create_mirrored_wake(curr_data, grnd_ht)
        for j in range(max_time_steps-1):
            print("J", j)
            for bl_arr_index in range(int(len(curr_arrs)/2)):
#                 print('bl', bl_arr_index, bl_arr_index*2*pi/(len(curr_arrs)))
                sum1 = np.array([curr_arrs[bl_arr_index].compute_net_ind_vel(j, zet_ind, curr_arrs) for zet_ind in range(len(curr_arrs[bl_arr_index].get_vortex_array(j)))])
                #print(np.array(prev_arrs[bl_arr_index].get_vortex_array(j)), (time_step*(free_stream_vel+sum1)).shape)
                new_poses = time_step*(free_stream_vel+sum1)
                #print(new_poses)
                for k in range(len(curr_arrs[bl_arr_index].get_vortex_array(j))):
                    #print(new_poses[k,:] + curr_arrs[bl_arr_index].get_filament(j,k).position)
                    new_poses[-(k+1),:] += curr_arrs[bl_arr_index].get_filament(j,k).position
                curr_arrs[bl_arr_index].add_vortex_array(curr_arrs[bl_arr_index].get_copy().get_vortex_array(j))
                curr_arrs[bl_arr_index].update_array_poses(j+1, new_poses)
                curr_arrs[bl_arr_index].add_filamnet_at_tip(blade_radius, j+1, bl_arr_index*2*pi/(len(curr_arrs))+omega*(j+1)*time_step)
                curr_arrs[int(len(curr_arrs)/2)+bl_arr_index].add_vortex_array(create_mirrored_vort_arr(curr_arrs[bl_arr_index].get_copy().get_vortex_array(j+1), grnd_ht))
#                 print('end', len(curr_arrs[bl_arr_index].vortex_array))
        '''for bl in range(len(curr_arrs)):
            print('BL ', bl)
            for k in range(len(curr_arrs[bl].get_vortex_array(0))):
                print(curr_arrs[bl].get_filament(0,k).position, curr_arrs[bl].get_filament(1,k).position)'''

    
    
def rms_cov_check(data):
    pass


num_fil=48
def make_init_vort_array(vorticity, vortex_core_radius, num_blades=4, theta_twist=-13, C_T=0.0050351, num_fil=num_fil):
    pos_fil, fil_vec = generate_prescribed_wake_vortex_array(theta_twist, C_T,num_fil+1)
    pos_fil = pos_fil[::-1,:]
    fil_vec = fil_vec[::-1,:]
    #print(pos_fil)
    v_arr = FullVortexArray2(num_fil)
    v_arr.add_new_psi()
    for fil_id in range(num_fil):
        v_arr.add_vortex_filament(VortexFilament(vorticity, pos_fil[fil_id,:], fil_vec[fil_id,:], vortex_core_radius),0)
    delta_theta = 2*np.pi/num_blades
    #print('hiii',len(rotate_vortex_array(v_arr, 0*delta_theta).vortex_array[0]), rotate_vortex_array(v_arr, 0*delta_theta).max_filaments)
    blades_v_arr = [rotate_vortex_array(v_arr, i*delta_theta) for i in range(num_blades)]
    return blades_v_arr
    
    

In [5]:
def run_algo():
    vorticity = 2.8
    vortex_core_radius = 0.0635 # in meters  
    omega = 40 # in rads/sec
    blade_radius = 0.8
    free_stream_vel = 0 
    update_fn = free_wake_update # free_wake_update, free_wake_with_grnd_update
    conv_check_fn = lambda a: False
    time_conv_check_fn = lambda a: False
    time_step = (24* np.pi/180)/omega 
    max_time_steps = 100
    init_arr = make_init_vort_array(vorticity, vortex_core_radius)  
    #print([init_arr[0].vortex_array[0][k].position for k in range(10)])
    result = solve_free_wake(init_arr, omega, blade_radius, free_stream_vel, update_fn, conv_check_fn, time_conv_check_fn, time_step, max_time_steps=20, max_iters=4, grnd_ht=0.8)
    
    return(result)
    
'''
Normal-
num_fill = 48
time_step = 24
vishnu's code phi_v max - 6*pi
WithGround-
num_fill = 24
time_step = 30
vishnu's code phi_v max - 4*pi
'''

"\nNormal-\nnum_fill = 48\ntime_step = 24\nvishnu's code phi_v max - 6*pi\nWithGround-\nnum_fill = 24\ntime_step = 30\nvishnu's code phi_v max - 4*pi\n"

In [6]:
data_alg = run_algo()

2.5506088558816193 2.157727648134868
Iteration no.  0
Iteration no.  1
Iteration no.  2
Iteration no.  3


In [7]:
#data_alg
#[[data_alg[0].vortex_array[j][k].position for k in range(20)] for j in range(20)]
#[data_alg[0].vortex_array[k][0] for k in range(10)]

In [8]:
data_L.append(data_alg)
data_L

[[<__main__.FullVortexArray2 at 0x1ce9634a2c8>,

In [9]:
vortices = data_L[-1][1].vortex_array
positions = np.array([[fill.position for fill in va] for va in vortices])
positions.shape

(20, 48, 3)

In [10]:
# filaments, co-ordinates, time 
positions2 = np.array([[[positions[time_idx,k_idx,c] for time_idx in range(20)]for c in range(3)]for k_idx in range(num_fil)])

In [11]:
%matplotlib notebook

# def generate_point_path(length, dims=2, num_points=2):
#     """    
#     Create a point and its path

#     Args:
#         length : time for which the point travels
#         dims (int, optional): number of dimensions the point has. Defaults to 2.

#     Returns:
#         np.array: size -> [dims,length]
#     """
#     dataset = []
#     for index in range(num_points):
#         pointData = np.empty((dims, length))
#         # x data
#         gamma = lambda t: 0.78 + (1 - 0.78) * np.exp(-0.145 * t)
#         pointData[0, :] = [
#             Helix.radius
#             * gamma(t)
#             * np.cos(index * 2 * np.pi / num_points + t * Helix.omega)
#             for t in range(length)
#         ]
#         # y data
#         pointData[1, :] = [
#             Helix.radius
#             * gamma(t)
#             * np.sin(index * 2 * np.pi / num_points + t * Helix.omega)
#             for t in range(length)
#         ]
#         # z data
#         pointData[2, :] = [(0 - t * Helix.zVel) for t in range(length)]

#         dataset.append(pointData)

#     return np.array(dataset)


def update_points(num, dataPoints, points):
    for point, data in zip(points, dataPoints):
        # NOTE: there is no .set_data() for 3 dim data...
        point.set_data(data[0:2, num])
        point.set_3d_properties(data[2, num])
    return points


# Attaching 3D axis to the figure
fig = plt.figure()
ax = axes3d.Axes3D(fig)

# Generate 3-D points
# length = 60
# num_points = 4
data = positions2

# Plots
# Make data.
X = np.linspace(-1,1,4)
Y = np.linspace(-1,1,4)
X, Y = np.meshgrid(X, Y)
Z = -0.8*np.ones(X.shape)

# Plot the surface.
# surf = ax.plot_surface(X, Y, Z,alpha=0.3)

# NOTE: Can't pass empty arrays into 3d version of plot(); for marker="o"
points = [
    ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1], marker="o", c="r")[0]
    for dat in data
]

blade_root = ax.plot(0,0,0,"og")[0]

rotor_axis = ax.plot(
    [0, 0],
    [0, 0],
    [0, - 0.8],
    "--g",
)[0]

# filaments, co-ordinates, time 

# Setting the axes properties
ax.set_xlim3d([-1.0, 1.0])
ax.set_xlabel("X")

ax.set_ylim3d([-1.0, 1.0])
ax.set_ylabel("Y")

ax.set_zlim3d([-1.0, 1.0])
ax.set_zlabel("Z")

ax.set_title("Vorticies")

# Creating the Animation object
point_ani = animation.FuncAnimation(
    fig, update_points, 20, fargs=(data, points), interval=1000, blit=True, repeat=True
)

SAVE = False
if SAVE == True:
    writer = animation.FFMpegWriter(fps=1)
    point_ani.save(
        "D:/nakul-pavilion/Google Drive/IITB/SS/Sem 6/AE667 (Rotary Wing Aerodynamics)/Project/free_wake_evolution.mov",
        writer=writer,
    )

plt.show()

<IPython.core.display.Javascript object>

TypeError: object of type 'int' has no len()